In [1]:
import cv2
import numpy as np
import os 
import PIL
from matplotlib import pyplot as plt

In [2]:
!pip install transformers
# Load model directly
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("mask-generation", model="ahishamm/skinsam")



/home/lorenzo/miniconda3/envs/tcc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
modelmask_folder = './pad/train/mask/melanoma/'
real_images_folder = './pad/train/data/melanoma/'
result_folder = './result_sam/' 
#get the first image from mask folder, then get the corresponding image from real_images_folder, then generate the mask and save the image with the mask on at result_folder
image_name = os.listdir(real_images_folder)[3]

# Read the image from the path
# image = cv2.imread(real_images_folder + image_name)
image_path = real_images_folder + image_name
image = PIL.Image.open(image_path)
#if image size is greater than 1920 1080, resize it
if image.size[0] > 1920 or image.size[1] > 1080:
    image = image.resize((1920, 1080))
    
# Generate segmentation mask
mask = pipe(image)

In [4]:
def apply_mask(image, mask, color=None):
    # Convert the mask to a 3 channel image
    if color is None:
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    else:
        mask_rgb = np.zeros_like(image)
        mask_rgb[mask>0] = color

    # Overlay the mask and image
    overlay_image = cv2.addWeighted(image, 0.7, mask_rgb, 0.3, 0)

    return overlay_image



In [5]:
masks = mask['masks']

In [6]:
import cv2
import numpy as np

def apply_masks_to_image(image_path, masks):
    # Load the image
    image = cv2.imread(image_path)
    #write original image
    cv2.imwrite(f'original_image.jpg', image)

    for i, mask in enumerate(masks):
        # Ensure the mask is a boolean array
        mask = np.array(mask, dtype=bool)

        # Apply the mask to the image
        masked_image = np.zeros_like(image)
        masked_image[mask] = image[mask]

        # Save the image
        cv2.imwrite(f'masked_image_{i}.jpg', masked_image)
apply_masks_to_image(image_path, masks)